# used to score base data
contains threshold #1 (different from T1, which was based on older models that included used two wheeler)

#used initially to just score, disregarded risk classes bc no thresholding yet
#upon threshold adjustment, used to score base data and for base data performance analysis

# Usable notebook
- Data must adhere to the required format;

### Change the datafile_name
e.g. "my_file.csv" ;<br>the .csv should be included in the name 

In [1]:
datafile_name = "EDA_df.csv"

### RUN ALL CELLS

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder

In [3]:
def standard_format_data(raw_datafilename):
    #load necessary libraries
    import pandas as pd
    import numpy as np
    
    #load data
    datafile_name = raw_datafilename
    data_filepath = f'./Raw_Data/{datafile_name}'
    basis_df = pd.read_csv(data_filepath)

    #lower case all column names
    basis_df.columns = basis_df.columns.str.lower()

    #choose necessary model variables
    variables = ['bureau_score', 'product_description','loan_account_no',
             'res_years_at_current_city','res_resident_status',
             'res_city','res_state','age',
             'manufacturer','down_payment_amount','total_income',
             'branch_name','occupation_type', 'industry',
             'interest_rate','number_of_installments','loan_asset_cost',
             'asset_model','channel','res_years_at_current_address',
             'gender', 'marital_status', 'nature_of_business',
             'years_in_occupation']
    #note: no requested_loam_amount (to be confirmed)
    df = basis_df[variables].copy()

    #for training only
    df = df[df['product_description'] != 'Used Two-Wheeler']
    df = df[df['total_income'] >= 1000]
    df = df[df['loan_account_no'] != 'TW01010103000002258833']

    #fill missing values; imputation using median values
    #cols cannot have missing values: bureau_score
    df['total_income'] = df['total_income'].fillna(df.total_income.median())
    df['monthly_income'] = df['total_income'] / 12 #derived
    df['down_payment_amount'] = df['down_payment_amount'].fillna(df.down_payment_amount.median())
    df['loan_asset_cost'] = df['loan_asset_cost'].fillna(df.loan_asset_cost.median())
    df['dp_ratio'] = df.down_payment_amount/ df.loan_asset_cost #derived

    df['res_resident_status'] = df.res_resident_status.fillna(df.res_resident_status.mode()[0])
    df['res_city'] = df['res_city'].fillna(df.res_city.mode()[0])
    df['occupation_type'] = df['occupation_type'].fillna(df.occupation_type.mode()[0])
    df['industry'] = df['industry'].fillna(df.industry.mode()[0])
    df['nature_of_business'] = df['nature_of_business'].fillna(df.nature_of_business.mode()[0])
    df['years_in_occupation'] = df['years_in_occupation'].fillna(df.years_in_occupation.median())
    
    df['branch_name'] = df['branch_name'].fillna(df.branch_name.mode()[0])
    df['age'] = df['age'].fillna(df.age.median())
    df['number_of_installments'] = df['number_of_installments'].fillna(df.number_of_installments.median())
    df['res_state'] = df['res_state'].fillna(df.res_state.mode()[0])
    df['product_description'] = df['product_description'].fillna(df.product_description.mode()[0])
    df['marital_status'] = df['marital_status'].fillna(df.marital_status.mode()[0])
    df['manufacturer'] = df['manufacturer'].fillna(df.manufacturer.mode()[0])
    
    #standardize values capitalization
    df = df.map(lambda x: x.upper() if isinstance(x, str) else x)
    
    #remove trailing and leading spaces
    obj_cols = list(df.select_dtypes('object'))
    df[obj_cols] = df[obj_cols].apply(lambda category: category.str.strip()) 
    
    #remove spaces in between values, replace with underscore
    df = df.map(lambda x: '_'.join(x.split()) if isinstance(x, str) else x)
    
    #convert to expected numerical datatypes
    float_cols = ['age','dp_ratio', 'down_payment_amount', 
                  'total_income','interest_rate','monthly_income']
    int_cols = ['bureau_score','res_years_at_current_city', 'loan_asset_cost','number_of_installments',
               'res_years_at_current_address','years_in_occupation']
    
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float)
    standard_df = df.copy()

    return standard_df

In [4]:
#clean column name of loan_account_no and standardize capitalization
def standard_format_raw(raw_datafilename):
    
    #load data
    datafile_name = raw_datafilename
    data_filepath = f'./Raw_Data/{datafile_name}'
    basis_df = pd.read_csv(data_filepath)

    #lower case all column names
    basis_df.columns = basis_df.columns.str.lower()
    
    df = basis_df.copy()
    df = df[df['product_description'] != 'Used Two-Wheeler']
    df = df[df['total_income'] >= 1000]
    df = df[df['loan_account_no'] != 'TW01010103000002258833']

    #standardize values capitalization
    df['loan_account_no'] = df['loan_account_no'].map(lambda x: x.upper() if isinstance(x, str) else x)
    
    #remove trailing and leading spaces
    obj_cols = list(df.select_dtypes('object'))
    
    standard_raw_df = df.copy()

    return standard_raw_df

In [5]:
standard_data = standard_format_data(datafile_name)
#standard_data.to_csv(f'./Input_data/01 Cleaned_Data/cleaned_{datafile_name}')

In [6]:
standard_data.years_in_occupation.median()

3.0

In [7]:
standard_data.nature_of_business.mode()

0    SERVICES
Name: nature_of_business, dtype: object

In [8]:
standard_data.shape

(49681, 26)

In [9]:
standard_data.columns

Index(['bureau_score', 'product_description', 'loan_account_no',
       'res_years_at_current_city', 'res_resident_status', 'res_city',
       'res_state', 'age', 'manufacturer', 'down_payment_amount',
       'total_income', 'branch_name', 'occupation_type', 'industry',
       'interest_rate', 'number_of_installments', 'loan_asset_cost',
       'asset_model', 'channel', 'res_years_at_current_address', 'gender',
       'marital_status', 'nature_of_business', 'years_in_occupation',
       'monthly_income', 'dp_ratio'],
      dtype='object')

In [10]:
raw_standard = standard_format_raw(datafile_name)
raw_all_cols = raw_standard.columns.tolist()
raw_all_cols.remove('loan_account_no')

In [11]:
#define a function that wrangles the base standard data
#i.e. grouping categories (crude feature engineering based on IV analysis)

def load_clean_data2(standard_df, segment):
    #load data
    df = standard_df

    #add new target columns
    # target_filename =  'tagged_12months_v2.csv'
    # target_filepath = f'./Raw_Data/{target_filename}'
    
    # new_col12m = pd.read_csv(target_filepath)
    # new_col12m = new_col12m.rename(columns={'loan_customer_no':'loan_account_no'})

    # merged_df_12m = pd.merge(basis_df, new_col12m, on='loan_account_no')
    # full_df = merged_df_12m.copy()

    # df = full_df.copy()
    #variable adjustments
    
    #manufacturer - Jan ok
    df['manufacturer_grp'] = np.where(
        (df['manufacturer'] == 'BAJAJ_AUTO') | (df['manufacturer'] == 'YAMAHA')
        | (df['manufacturer'] == 'APRILIA') | (df['manufacturer'] == 'ROYAL_ENFIELD')
        | (df['manufacturer'] == 'PIAGGIO') | (df['manufacturer'] == 'KTM')
        | (df['manufacturer'] == 'JAWA'),
        'OTHERS', df['manufacturer'])
    
    #channel - Jan ok
    df['channel_grp'] = np.where(
        (df['channel'] == 'ALTERNATE_CHANNEL') | (df['channel'] == 'BRANCH')
        | (df['channel'] == 'DIRECT')| (df['channel'] == 'REFERRAL'),
        'NON_DEALER', df['channel'])

    #industry - Jan ok
    df['industry_grp'] = np.where(
        (df['industry'] != 'APPARELS') & (df['industry'] != 'AGRICULTURE'),
        'OTHERS',df['industry'])

    #branch_name - Jan ok
    df['branch_grp'] = np.where(
        (df['branch_name'] == 'EAST_BANGALORE') | (df['branch_name'] == 'SOUTH_DELHI') |
        (df['branch_name'] == 'NORTH_BANGALORE') | (df['branch_name'] == 'EAST_DELHI') |
        (df['branch_name'] == 'NOIDA')| (df['branch_name'] == 'GURGAON')|
        (df['branch_name'] == 'WEST_BANGALORE')| (df['branch_name'] == 'NORTH_DELHI')
        | (df['branch_name'] == 'FARIDABAD'),
            'OTHERS',df['branch_name'])

    #grouping asset_model - Jan ok
    text_list = ['SPLENDOR', 'ACCESS', 'BURGMAN', 'ACTIVA', 'NTORQ', 'JUPITER', 'HF', 'AVENIS']
    # Function to check and assign text or 'Others'
    def check_text(value):
        value = value.replace('_', ' ')  # Replace underscores with spaces
        for text in text_list:
            if text in value:
                return text
        return 'OTHERS'

    # Create new column based on the function
    df['asset_model_grp'] = df['asset_model'].apply(check_text)

    #res_city - Jan oK
    df['res_city_grp'] = np.where(
        (df['res_city'] != 'MUMBAI') & (df['res_city'] != 'THANE')
        & (df['res_city'] != 'BANGALORE')& (df['res_city'] != 'DELHI')
        & (df['res_city'] != 'GHAZIABAD'),
        'OTHERS',df['res_city'])

    #res_state - Jan ok
    df['res_state_grp'] = np.where(
        (df['res_state'] == 'HARYANA') | (df['res_state'] == 'LAKSHADWEEP'),
        'OTHERS',df['res_state'])

    #res_resident_status - Jan ok
    df['res_resident_status_grp'] = np.where(
        (df['res_resident_status'] == 'OWNED_BY_PARENT_/SIBLING') | (df['res_resident_status'] == 'OWNED_BY_SELF/SPOUSE')
        , 'OWNED', df['res_resident_status'])

    df['res_resident_status_grp'] = np.where(
        (df['res_resident_status'] == 'COMPANY_PROVIDED') | (df['res_resident_status'] == 'RENTED')
        | (df['res_resident_status'] == 'HOSTEL')| (df['res_resident_status'] == 'PAYING_GUEST')
        , 'RENTED', df['res_resident_status_grp'])

    #branch_name
    df['nature_of_business'] = np.where(
        (df['nature_of_business'] != 'SERVICES') & (df['nature_of_business'] != 'AGRICULTURE'),
            'OTHERS',df['nature_of_business'])

    working_df = df.copy()
    working_df = working_df.drop(columns = ['manufacturer', 'channel','industry',
                                            'branch_name', 'asset_model','res_city',
                                            'res_state','res_resident_status'])

    #dfs
    if segment == 'NTC':
        df_NTC = working_df[working_df['bureau_score'] < 300]
        df_NTC = df_NTC.drop(columns = ['bureau_score'])
        return df_NTC
    
    elif segment == 'low_CIBIL':
        df_low_CIBIL = working_df[(working_df['bureau_score'] >= 300) & (working_df['bureau_score'] <= 730)]
        df_low_CIBIL = df_low_CIBIL.drop(columns = ['bureau_score'])
        return df_low_CIBIL
    
    elif segment == 'mid_CIBIL':
        df_mid_CIBIL = working_df[(working_df['bureau_score'] >= 731) & (working_df['bureau_score'] <= 749)]
        df_mid_CIBIL = df_mid_CIBIL.drop(columns = ['bureau_score'])
        return df_mid_CIBIL
    
    elif segment == 'high_CIBIL':
        df_high_CIBIL = working_df[working_df['bureau_score'] >= 750]
        df_high_CIBIL = df_high_CIBIL.drop(columns = ['bureau_score'])
        return df_high_CIBIL
        
    else:
        return ('Choose among NTC, low_CIBIL, mid_CIBIL, high_CIBIL')

In [23]:
wrangled_df_NTC = load_clean_data2(standard_data, 'NTC')
wrangled_df_low_CIBIL = load_clean_data2(standard_data, 'low_CIBIL')
wrangled_df_mid_CIBIL = load_clean_data2(standard_data, 'mid_CIBIL')
wrangled_df_high_CIBIL = load_clean_data2(standard_data, 'high_CIBIL')

# wrangled_df_NTC.to_csv(f'./Input_data/02 Wrangled_Data/wrangled_NTC_{datafile_name}',index=False)
# wrangled_df_low_CIBIL.to_csv(f'./Input_data/02 Wrangled_Data/wrangled_low_CIBIL_{datafile_name}',index=False)
# wrangled_df_mid_CIBIL.to_csv(f'./Input_data/02 Wrangled_Data/wrangled_mid_CIBIL_{datafile_name}',index=False)
# wrangled_df_high_CIBIL.to_csv(f'./Input_data/02 Wrangled_Data/wrangled_high_CIBIL_{datafile_name}',index=False)

In [24]:
print(wrangled_df_NTC.shape)
print(wrangled_df_low_CIBIL.shape)
print(wrangled_df_mid_CIBIL.shape)
print(wrangled_df_high_CIBIL.shape)

(18278, 25)
(11369, 25)
(7425, 25)
(12609, 25)


In [12]:
18278 + 11369 + 7425 + 12609

49681

# Scoring

## NTC

In [13]:
#lgbm
#load model for ntc
with open('./Refreshed_models/lgb_modelr_ntc.pkl', 'rb') as model_file:
    NTC_model = pickle.load(model_file)

#load ohe for ntc
with open('./Refreshed_models/ohe_ntc.pkl', 'rb') as ohe_file:
    NTC_ohe = pickle.load(ohe_file)

In [14]:
def risk_class_NTC(x):
    if x <= 0.339905769192112:
        return 'Low Risk'
    elif 0.339905769192112 < x <= 0.489970455472209:
        return 'Middle Risk'
    elif 0.489970455472209 < x <= 0.599997439305038:
        return 'High Risk'
    else:
        return 'Very High Risk'

In [15]:
#for lgbm
X_NTC = wrangled_df_NTC.drop(columns = ["loan_account_no"])

if not X_NTC.empty:
    object_columns = X_NTC.select_dtypes(include=['object','category']).columns
    one_hot_encoded_object_X = NTC_ohe.transform(X_NTC[object_columns])
    final_X = pd.concat([one_hot_encoded_object_X, X_NTC.select_dtypes('number')], axis=1)
    
    y_pred_proba = NTC_model.predict_proba(final_X)[:,1]
    
    wrangled_df_NTC['probability'] = y_pred_proba
    wrangled_df_NTC['Risk Class'] = wrangled_df_NTC['probability'].apply(risk_class_NTC)
    scored_wrangled_df_NTC = wrangled_df_NTC.copy()
else:
    scored_wrangled_df_NTC = X_NTC.copy()

## low_CIBIL

In [16]:
#load model for low_CIBIL
with open('./Refreshed_models/lgb_modelr_low_CIBIL.pkl', 'rb') as model_file:
    low_CIBIL_model = pickle.load(model_file)

#load ohe for low_CIBIL
with open('./Refreshed_models/ohe_low_CIBIL.pkl', 'rb') as ohe_file:
    low_CIBIL_ohe = pickle.load(ohe_file)

In [17]:
def risk_class_low_CIBIL(x):
    if x <= 0.329906626610688:
        return 'Low Risk'
    elif 0.329906626610688 < x <= 0.479921138916135:
        return 'Middle Risk'
    elif 0.479921138916135 < x <= 0.609980671913991:
        return 'High Risk'
    else:
        return 'Very High Risk'

In [18]:
X_low_CIBIL = wrangled_df_low_CIBIL.drop(columns = ["loan_account_no"])

if not X_low_CIBIL.empty:
    object_columns = X_low_CIBIL.select_dtypes(include=['object','category']).columns
    one_hot_encoded_object_X = low_CIBIL_ohe.transform(X_low_CIBIL[object_columns])
    final_X = pd.concat([one_hot_encoded_object_X, X_low_CIBIL.select_dtypes('number')], axis=1)
    
    y_pred_proba = low_CIBIL_model.predict_proba(final_X)[:,1]
    
    wrangled_df_low_CIBIL['probability'] = y_pred_proba
    wrangled_df_low_CIBIL['Risk Class'] = wrangled_df_low_CIBIL['probability'].apply(risk_class_low_CIBIL)
    scored_wrangled_df_low_CIBIL = wrangled_df_low_CIBIL.copy()
else:
    scored_wrangled_df_low_CIBIL = X_low_CIBIL.copy()

## mid_CIBIL

In [19]:
#load model for mid_CIBIL
with open('./Refreshed_models/rf_model_mid_CIBIL.pkl', 'rb') as model_file:
    mid_CIBIL_model = pickle.load(model_file)

In [20]:
def risk_class_mid_CIBIL(x):
    if x <= 0.0459969782663126:
        return 'Low Risk'
    elif 0.0459969782663126 < x <= 0.0699938217029362:
        return 'Middle Risk'
    elif 0.0699938217029362 < x <= 0.0829890704783013:
        return 'High Risk'
    else:
        return 'Very High Risk'

In [21]:
#this is for RF
X_mid_CIBIL = wrangled_df_mid_CIBIL.drop(columns = ["loan_account_no"])

if not X_mid_CIBIL.empty:
    y_pred_proba = mid_CIBIL_model.predict_proba(X_mid_CIBIL)[:,1]

    wrangled_df_mid_CIBIL['probability'] = y_pred_proba
    wrangled_df_mid_CIBIL['Risk Class'] = wrangled_df_mid_CIBIL['probability'].apply(risk_class_mid_CIBIL)
    scored_wrangled_df_mid_CIBIL = wrangled_df_mid_CIBIL.copy()
else:
    scored_wrangled_df_mid_CIBIL = X_mid_CIBIL.copy()

## high_CIBIL

In [25]:
#load model for high_CIBIL
with open('./Refreshed_models/xgbc_modelr_high_CIBIL.pkl', 'rb') as model_file:
    high_CIBIL_model = pickle.load(model_file)

#load ohe for high_CIBIL
with open('./Refreshed_models/ohe_high_CIBIL.pkl', 'rb') as ohe_file:
    high_CIBIL_ohe = pickle.load(ohe_file)

In [14]:
high_CIBIL_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7200762468698007, colsample_bynode=None,
              colsample_bytree=0.6488152939379115, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eta=0.10903538202225403, eval_metric=None, feature_types=None,
              gamma=0.5158278167282759, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.27280612062363463,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=50, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=409, n_jobs=-1,
              num_parallel_tree=None, ...)

In [26]:
def risk_class_high_CIBIL(x):
    if x <= 0.0359869040548801:
        return 'Low Risk'
    elif 0.0359869040548801 < x <= 0.071941502392292:
        return 'Middle Risk'
    elif 0.071941502392292 < x <= 0.0939865708351135:
        return 'High Risk'
    else:
        return 'Very High Risk'

In [27]:
X_high_CIBIL = wrangled_df_high_CIBIL.drop(columns = ["loan_account_no"])

if not X_high_CIBIL.empty:
    object_columns = X_high_CIBIL.select_dtypes(include=['object','category']).columns
    one_hot_encoded_object_X = high_CIBIL_ohe.transform(X_high_CIBIL[object_columns])
    final_X = pd.concat([one_hot_encoded_object_X, X_high_CIBIL.select_dtypes('number')], axis=1)
    
    y_pred_proba = high_CIBIL_model.predict_proba(final_X)[:,1]
    
    wrangled_df_high_CIBIL['probability'] = y_pred_proba
    wrangled_df_high_CIBIL['Risk Class'] = wrangled_df_high_CIBIL['probability'].apply(risk_class_high_CIBIL)
    scored_wrangled_df_high_CIBIL = wrangled_df_high_CIBIL.copy()
else:
    scored_wrangled_df_high_CIBIL = X_high_CIBIL.copy()

In [31]:
high_CIBIL_model.predict_proba(final_X)[:,1].max()

0.45177943

In [21]:
high_CIBIL_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7200762468698007, colsample_bynode=None,
              colsample_bytree=0.6488152939379115, device=None,
              early_stopping_rounds=None, enable_categorical=True,
              eta=0.10903538202225403, eval_metric=None, feature_types=None,
              gamma=0.5158278167282759, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.27280612062363463,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=50, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=409, n_jobs=-1,
              num_parallel_tree=None, ...)

In [25]:
df_scored = pd.concat([scored_wrangled_df_NTC, scored_wrangled_df_low_CIBIL,
                              scored_wrangled_df_mid_CIBIL,scored_wrangled_df_high_CIBIL])

scores = df_scored[['loan_account_no', 'probability', 'Risk Class']]
final_frame = raw_standard.merge(scores, how='inner', on='loan_account_no')
scores_temp = final_frame[['loan_account_no','bureau_score' ,'probability', 'Risk Class']]
#re-order columns
column_names = raw_all_cols
column_names.insert(0, 'loan_account_no')
column_names.append('probability')
column_names.append('Risk Class')

final_frame = final_frame.reindex(columns=column_names)
#final_frame.to_csv(f'./Output_data/scored_{datafile_name}', index=False)

In [26]:
df_scored.columns

Index(['product_description', 'loan_account_no', 'res_years_at_current_city',
       'age', 'down_payment_amount', 'total_income', 'occupation_type',
       'interest_rate', 'number_of_installments', 'loan_asset_cost',
       'res_years_at_current_address', 'gender', 'marital_status',
       'nature_of_business', 'years_in_occupation', 'monthly_income',
       'dp_ratio', 'manufacturer_grp', 'channel_grp', 'industry_grp',
       'branch_grp', 'asset_model_grp', 'res_city_grp', 'res_state_grp',
       'res_resident_status_grp', 'probability', 'Risk Class'],
      dtype='object')

In [27]:
def cibil_group(x):
    if x < 300:
        return 'NTC'
    elif 300 <= x <= 730:
        return 'low_CIBIL'
    elif 731 <= x <= 749:
        return 'mid_CIBIL'
    else:
        return 'high_CIBIL'

scores_temp['CIBIL_group'] = scores_temp['bureau_score'].apply(cibil_group)


In [28]:
target_filename =  'tagged_12months_v2.csv'
target_filepath = f'./Raw_Data/{target_filename}'

new_col12m = pd.read_csv(target_filepath)
new_col12m = new_col12m.rename(columns={'loan_customer_no':'loan_account_no'})

merged_thresholding_base = pd.merge(scores_temp, new_col12m, on='loan_account_no')
final_thresholding_base = merged_thresholding_base.copy()

In [29]:
final_thresholding_base.to_csv(f'temp_{datafile_name}_thresholding_v0.csv', index=False)

In [30]:
#scores.to_csv(f'{datafile_name}_risk_tags.csv', index=False)

In [31]:
#scores.to_csv(f'./Output_data/{datafile_name}_risk_tags.csv', index=False)

In [32]:
#final_frame.to_csv(f'./Output_data/comp_results_{datafile_name}', index=False)

In [19]:
scored_wrangled_df_high_CIBIL.probability.max()

0.45177943

In [20]:
scored_wrangled_df_high_CIBIL.probability.min()

0.001060352